In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [107]:
# Load the data
url= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv"
df = pd.read_csv(url)
df.sample(5)

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
213,2014,CHEVROLET,EXPRESS 1500 CARGO AWD,VAN - CARGO,5.3,8,A4,X,18.3,14.2,16.5,17,380
864,2014,NISSAN,XTERRA 4WD,SUV - SMALL,4.0,6,M6,X,15.2,11.9,13.7,21,315
727,2014,MERCEDES-BENZ,C 350 4MATIC,COMPACT,3.5,6,AS7,Z,11.9,8.6,10.4,27,239
88,2014,BMW,335i xDRIVE,COMPACT,3.0,6,A8,Z,11.7,7.8,9.9,29,228
134,2014,BMW,X6 xDRIVE50i,SUV - STANDARD,4.4,8,A8,Z,16.4,11.3,14.1,20,324


In [108]:
# describe the statistical data
df.describe()

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
count,1067.0,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000
mean,2014.0,3.346298,5.794752,13.296532,9.474602,11.580881,26.441425,256.228679
std,0.0,1.415895,1.797447,4.101253,2.794510,3.485595,7.468702,63.372304
min,2014.0,1.000000,3.000000,4.600000,4.900000,4.700000,11.000000,108.000000
25%,2014.0,2.000000,4.000000,10.250000,7.500000,9.000000,21.000000,207.000000
50%,2014.0,3.400000,6.000000,12.600000,8.800000,10.900000,26.000000,251.000000
75%,2014.0,4.300000,8.000000,15.550000,10.850000,13.350000,31.000000,294.000000
max,2014.0,8.400000,12.000000,30.200000,20.500000,25.800000,60.000000,488.000000


In [109]:
# describe more info about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MODELYEAR                 1067 non-null   int64  
 1   MAKE                      1067 non-null   object 
 2   MODEL                     1067 non-null   object 
 3   VEHICLECLASS              1067 non-null   object 
 4   ENGINESIZE                1067 non-null   float64
 5   CYLINDERS                 1067 non-null   int64  
 6   TRANSMISSION              1067 non-null   object 
 7   FUELTYPE                  1067 non-null   object 
 8   FUELCONSUMPTION_CITY      1067 non-null   float64
 9   FUELCONSUMPTION_HWY       1067 non-null   float64
 10  FUELCONSUMPTION_COMB      1067 non-null   float64
 11  FUELCONSUMPTION_COMB_MPG  1067 non-null   int64  
 12  CO2EMISSIONS              1067 non-null   int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 108.5+ KB


In [110]:
# Your task will be to create a simple linear regression model from one of these features to
#  predict CO2 emissions of unobserved cars based on that feature.

# OBSERVATIONS:
# MODELYEAR has 0 standard deviation, and thus has no interesting information content.
# The max in FUELCONSUMPTION_CITY is much much higher than the 75% so this means this could be and OUTLIER


In [ ]:
# PREPROCESSING

# drop the model year since its std is 0 thus it is not interesting
# drop the MAKE, MODEL and VEHICLECLASS cause they seem to not be related to the targeted output logically
df.drop(columns= ["MODELYEAR", "MAKE", "MODEL", "VEHICLECLASS", "TRANSMISSION"], inplace = True)

# now label encode the FUELTYPE
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
fuelTypeEncoded = pd.DataFrame(le.fit_transform(df["FUELTYPE"].values), columns=["FUELTYPEENCODED"])

df = pd.concat([df, fuelTypeEncoded], axis = 1)
df.sample(5)

,ENGINESIZE,CYLINDERS,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS,FUELTYPEENCODED
260,1.2,4.0,X,7.6,6.0,6.9,41.0,159.0,NaN
628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
962,2.0,4.0,X,9.5,7.2,8.5,33.0,196.0,NaN
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
